In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectFromModel

In [2]:
df = pd.read_csv("preprocessed_data.csv")
df.head()

,rooms,floor,floorCount,latitude,longitude,poiCount,schoolDistance,clinicDistance,postOfficeDistance,kindergartenDistance,...,city_lodz,city_lublin,city_poznan,city_radom,city_rzeszow,city_szczecin,city_warszawa,city_wroclaw,ownership_cooperative,ownership_udział
0,4.0,3.0,4.0,1.094088,-2.820267,-0.816073,1.487393,-0.026132,0.230208,1.028105,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2.0,5.0,5.0,1.092505,-2.781375,1.647109,-0.585392,-0.607703,-0.383685,-0.380457,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,5.0,4.0,4.0,1.102938,-2.777586,-0.440334,-0.432099,-0.410059,0.479928,-0.185681,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,3.0,7.0,7.0,1.106274,-2.779584,-0.440334,-0.316574,-0.228318,0.340501,-0.233188,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3.0,4.0,4.0,1.112952,-2.765224,-0.565580,-0.603165,0.413455,0.860750,1.358273,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [3]:
X = df.drop(columns=["log_price"])
y = df["log_price"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
feature_names = X_train.columns.tolist()
print(feature_names)

['rooms', 'floor', 'floorCount', 'latitude', 'longitude', 'poiCount', 'schoolDistance', 'clinicDistance', 'postOfficeDistance', 'kindergartenDistance', 'restaurantDistance', 'collegeDistance', 'pharmacyDistance', 'hasParkingSpace', 'hasBalcony', 'hasElevator', 'hasSecurity', 'hasStorageRoom', 'squareMeters_capped', 'centreDistance_winsorized', 'location_cluster', 'type_encoded', 'age', 'city_bydgoszcz', 'city_czestochowa', 'city_gdansk', 'city_gdynia', 'city_katowice', 'city_krakow', 'city_lodz', 'city_lublin', 'city_poznan', 'city_radom', 'city_rzeszow', 'city_szczecin', 'city_warszawa', 'city_wroclaw', 'ownership_cooperative', 'ownership_udział']


## Model Tuning

In [6]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 500, 2000, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "max_depth": trial.suggest_int("max_depth", 5, 15),
        "num_leaves": trial.suggest_int("num_leaves", 20, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0)
    }
    model = lgb.LGBMRegressor(**params, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)
best_params = study.best_params

[I 2025-04-26 20:46:07,310] A new study created in memory with name: no-name-0813323e-fde2-42d5-a6c0-4e87b7593e42


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012782 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3347
[LightGBM] [Info] Number of data points in the train set: 33385, number of used features: 38
[LightGBM] [Info] Start training from score 13.507787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2025-04-26 20:46:11,235] Trial 0 finished with value: 0.07196290548648807 and parameters: {'n_estimators': 600, 'learning_rate': 0.09101944530615423, 'max_depth': 14, 'num_leaves': 81, 'subsample': 0.9519875117641186, 'colsample_bytree': 0.7573675454740083}. Best is trial 0 with value: 0.07196290548648807.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004983 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3347
[LightGBM] [Info] Number of data points in the train set: 33385, number of used features: 38
[LightGBM] [Info] Start training from score 13.507787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[I 2025-04-26 20:46:13,225] Trial 1 finished with value: 0.08492994470393633 and parameters: {'n_estimators': 500, 'learning_rate': 0.05688520881209664, 'max_depth': 8, 'num_leaves': 68, 'subsample': 0.8800957269101718, 'colsample_bytree': 0.7999285185952795}. Best is trial 0 with value: 0.07196290548648807.


...

[I 2025-04-26 20:48:01,047] Trial 17 finished with value: 0.0675206692290518 and parameters: {'n_estimators': 1600, 'learning_rate': 0.045966339922049584, 'max_depth': 9, 'num_leaves': 90, 'subsample': 0.7925718134290933, 'colsample_bytree': 0.9985720208648836}. Best is trial 10 with value: 0.0613306714139275.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005285 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3347
[LightGBM] [Info] Number of data points in the train set: 33385, number of used features: 38
[LightGBM] [Info] Start training from score 13.507787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2025-04-26 20:48:05,152] Trial 18 finished with value: 0.06745741913741224 and parameters: {'n_estimators': 1000, 'learning_rate': 0.06952656086239098, 'max_depth': 15, 'num_leaves': 91, 'subsample': 0.6663268304604388, 'colsample_bytree': 0.5474273808881309}. Best is trial 10 with value: 0.0613306714139275.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004792 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3347
[LightGBM] [Info] Number of data points in the train set: 33385, number of used features: 38
[LightGBM] [Info] Start training from score 13.507787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2025-04-26 20:48:09,713] Trial 19 finished with value: 0.0687694483663155 and parameters: {'n_estimators': 1400, 'learning_rate': 0.08566874831166853, 'max_depth': 13, 'num_leaves': 48, 'subsample': 0.6808601138841142, 'colsample_bytree': 0.6541030821999934}. Best is trial 10 with value: 0.0613306714139275.


In [7]:
lgb_model = lgb.LGBMRegressor(**best_params, random_state=42)
lgb_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017256 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3347
[LightGBM] [Info] Number of data points in the train set: 33385, number of used features: 38
[LightGBM] [Info] Start training from score 13.507787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMRegressor(colsample_bytree=0.5604657651551779,
              learning_rate=0.07011703589938538, max_depth=15,
              n_estimators=1500, num_leaves=96, random_state=42,
              subsample=0.7556073847790201)

In [8]:
y_pred_lgb = lgb_model.predict(X_test)

In [9]:
mae = mean_absolute_error(y_test, y_pred_lgb)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
print('mae:',mae)
print('rmse:',rmse)

mae: 0.0613306714139275
rmse: 0.0937712707932223


In [10]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(lgb_model, f)